# IPO Performance
- Retrieves historical price performance
- Uses Yahoo Finance for historical price data
 - https://finance.yahoo.com/

In [1]:
import numpy as np
import pandas as pd

import nasdaq

from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
yf.pdr_override()

%matplotlib inline
import matplotlib.pyplot as plt
import time

In [2]:
# params
start_date = '2018-06-01'
ipo_performance_csv = '../Datasets/0 ipo listings.csv'

# Create NASDAQ IPO List

In [3]:
df_symbols = nasdaq.get_ipo_list(start_date)
print('symbols', df_symbols.shape[0])

date range: odict_keys(['2018-06', '2018-07'])
symbols 39


In [4]:
df_symbols.head()

,Company Name,Symbol,Market,Price,Shares,Offer Amount,Date Priced
Symbol,,,,,,,
DOMO,"DOMO, INC.",DOMO,NASDAQ Global,21.0,9200000,193200000.0,2018-06-29
BJ,"BJ'S WHOLESALE CLUB HOLDINGS, INC.",BJ,NYSE,17.0,37500000,637500000.0,2018-06-28
BV,"BRIGHTVIEW HOLDINGS, INC.",BV,NYSE,22.0,21300000,468600000.0,2018-06-28
ENTX,ENTERA BIO LTD.,ENTX,NASDAQ Capital,8.0,1400000,11200000.0,2018-06-28
EVER,"EVERQUOTE, INC.",EVER,NASDAQ Global,18.0,4687500,84375000.0,2018-06-28


In [5]:
#load previous to save time
df_prev = pd.read_csv(ipo_performance_csv).dropna()
df_prev.index = df_prev['Symbol']

#reduce by previously done
df_symbols = df_symbols.loc[df_symbols.index.difference(df_prev.index)]

# Add Historical Price Performance

- Use open prices only (retial traders won't get best price)

In [6]:
#target variables and trading days range
targets = {
    '1D' : 1,
    '1W' : 5,
    '1M' : 20,
    '3M' : 60
}

In [7]:
#add targets
for t in targets.keys():
    df_symbols[t] = np.NaN

#calculate targets    
counter = 0    
    
for x in df_symbols['Symbol']:
    try:
        time.sleep(5)
        counter += 1
        print('\n( ' + str(counter) + ' / ' + str(df_symbols.shape[0]) + ' ) ' + str(x))

        #get historical prices
        df = pdr.get_data_yahoo(x, start=start_date)

        #add targets
        for t in targets.keys():
            s = df.pct_change(periods=targets[t]).dropna().head(1)['Open']
            if df.head(1).index[0] == df_symbols.at[x, 'Date Priced'] and s.shape[0] > 0 and abs(s[0]) > 0:
                df_symbols.at[x, t] = s[0]
            else:
                df_symbols.at[x, t] = np.NaN
                
        #store
        df_full = pd.concat([df_symbols, df_prev], axis=0)
        #df_full.dropna(inplace=True)        
        df_full.to_csv(ipo_performance_csv, index=False)                
    except Exception as e:
        print(x, e)     

In [8]:
print('done', df_symbols.shape[0], 'prev', df_prev.shape[0])
df_symbols.head(3)

done 0 prev 996


,Company Name,Symbol,Market,Price,Shares,Offer Amount,Date Priced,1D,1W,1M,3M
Symbol,,,,,,,,,,,
